In [15]:
import pandas as pd
import os
from data_processing import steps, data, data_train, data_test, exog_vars, hour_in_week

cwd = os.path.abspath("")
backtest_result_file_path = os.path.join(cwd, './data/backtest_result.csv')

df = pd.read_csv(backtest_result_file_path)
df.index = data_test.index

In [2]:
import joblib
modelling_file_path = os.path.join(cwd, 'src/modelling/model/forecaster.joblib')
backtest_result_file_path = os.path.join(cwd, './data/backtest_result.csv')

forecaster = joblib.load(modelling_file_path)
regressor_name = str(forecaster.regressor).split('()')[0]

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.4.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator RandomForestRegressor from version 1.4.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator K

In [3]:
from forecast import BacktestingForecaster
bf = BacktestingForecaster(forecaster         = forecaster,
                          y                  = data['y'],
                          steps              = hour_in_week,
                          metric             = 'mean_absolute_percentage_error',
                          initial_train_size = len(data_train),
                          refit              = True,)
backtest_indexes = bf._create_backtesting_folds()

backtesting_folds_list = bf.get_backtesting_folds(backtest_indexes = backtest_indexes,
                         backtest_result_df = df,
                         regressor_name= regressor_name)

In [4]:
backtesting_folds_list[0]

,y,"RandomForestRegressor(max_depth=10, n_estimators=50)"
Datetime,,
2023-04-21 00:00:00+00:00,7800.6900,7814.054279
2023-04-21 01:00:00+00:00,7693.6300,7713.332677
2023-04-21 02:00:00+00:00,7859.2450,7793.479838
2023-04-21 03:00:00+00:00,8256.8700,8103.770917
2023-04-21 04:00:00+00:00,8987.8550,8687.300933
...,...,...
2023-04-27 18:00:00+00:00,9445.4600,9376.824041
2023-04-27 19:00:00+00:00,9242.1475,9158.445793
2023-04-27 20:00:00+00:00,9082.6250,8961.960734
